## **Type System in Julia** 

We are going to investigate the Julia Type and casting system in this file. 

There are 2 types of type in julia
* Types that are type by itself. 
* Types that are paramaterized by another type. 

When doing typing, we need to becareful about Parametric Types, how they works are important. 

* Subtypes 
* Type conversion 
* Parametric Types
* Paramaterized parametric types and subtypes. 
* THERE ARE NO TYPE PROMOTIONS IN JULIA
  * This is achieved via parametric types of function parameters, and, method overriding. 


Functions dispatch are based on **Subtyping**, no promotions or conversion is involved. Voided type declarations is default to type `Any`. 

In [14]:
# Hiearchy of the primitive types
display(Float64 <: Number)
display(Float64 <: Complex)
display(Int64 <: Float64)
display(Vector <: Array)

true

false

false

true

In [9]:
# Casting for Primitive Types: convert(targetType, instance)
x = convert(Int64, 3.0) # Successful 
x = convert(Int64, 3.5) # Not good

LoadError: InexactError: Int64(3.5)

In [1]:
# Parametric Type casting 
x = Array{Int64}(undef, 3, 3)
y = convert(Array{Float64}, x)
display(y)
z = convert(Array{Complex}, x)
# Don't use these method too much it's slow. 

3×3 Matrix{Float64}:
 4.87804e8  4.87805e8  4.53295e8
 4.87804e8  3.19098e8  4.52487e8
 4.87804e8  4.53428e8  4.88171e8

3×3 Matrix{Complex}:
 487804432+0im  487804528+0im  453295376+0im
 487804464+0im  319098416+0im  452487440+0im
 487804496+0im  453428336+0im  488171280+0im

In [2]:
# Parametric Composite Types, 
# the Inner Type must be equal then the outter type determine Hiearchy. 
display(Array{Int64} <: Array{Int64})
display(Vector{Int64} <: Array{Int64})



true

true

In [24]:
# Unequal inner types breaks typing for the container
display(Array{Int64} <: Array{Number})
display(Vector{Int64} <: Array{Int64})
display(Vector{Int64} <: Array{Number})
display(Vector{Int64} <: Array)
# Not even any can help
display(Vector{Int64} <: Array{Any})
display(Vector <: Array{Any})


false

true

false

true

false

false

**A Guess on the Rules here**

Compare the inner paramatric type first, if it's matching (Like exactly), then it will compare the outer container type. 

A voided parametric type is a pass for comparing the inner types. 

To make a parametric type that contains a lot of other parametric type, we need to paramaterized the paramatric type. 

In [30]:
function IsNumericalVector(arr::Vector{T}) where {T<: Number}
    println("Good, it's a numerical Vector")
end

IsNumericalVector (generic function with 1 method)

In [34]:
arr = Vector{Int64}(undef, 3)
IsNumericalVector(arr)
arr = Vector{Complex{Int64}}(undef, 3)
IsNumericalVector(arr)
arr = Vector{Array}(undef, 3) # Parametric type not a subtype of Number
IsNumericalVector(arr)


Good, it's a numerical Vector
Good, it's a numerical Vector


LoadError: MethodError: no method matching IsNumericalVector(::Vector{Array})
[0mClosest candidates are:
[0m  IsNumericalVector([91m::Vector{T}[39m) where T<:Number at In[30]:1

In [1]:
# One last thing: 
display(Array{Int64} <: Array{T} where {T<:Number})

display(Array{Int64} <: Array{<:Number})

true

true

In [5]:
# containers without parametric types
display(typeof(Array))
# The output is UnionAll it's because this is a constructor
dump(Array)

UnionAll

UnionAll
  var: TypeVar
    name: Symbol T
    lb: Union{}
    ub: Any
  body: UnionAll
    var: TypeVar
      name: Symbol N
      lb: Union{}
      ub: Any
    body: Array{T, N} <: DenseArray{T, N}


In [10]:
display(typeof([]))
# Empty vector has the potential to be anyelement
display(typeof([1.2, 3, 3]))

Vector{Any} (alias for Array{Any, 1})

Vector{Float64} (alias for Array{Float64, 1})

In [17]:
# There are NO AUTOMATIC PROMOTIONS IN JULIA

",1:3 "

### **Meta Programming and Types**

* `subtypes()` is a function that returns the subtype of a given type, it goes down by one leve. 


In [7]:
# a function that prints types hiearchy
function SubTypeTree(t, level=1, indent=4)
    level == 1 && println(t)
    for s in subtypes(t)
      println(join(fill(" ", level * indent)) * string(s))
      SubTypeTree(s, level+1, indent)
    end
end

SubTypeTree (generic function with 3 methods)

In [9]:
# Visualize all non-container types. 
SubTypeTree(Number)

Number
    Complex
    Real
        AbstractFloat
            BigFloat
            Float16
            Float32
            Float64
        AbstractIrrational
            Irrational
        Integer
            Bool
            Signed
                BigInt
                Int128
                Int16
                Int32
                Int64
                Int8
            Unsigned
                UInt128
                UInt16
                UInt32
                UInt64
                UInt8
        Rational
Complex{var"#s28"} where var"#s28"<:Number


In [10]:
SubTypeTree(AbstractArray)

AbstractArray
    AbstractRange
        LinRange
        OrdinalRange
            AbstractUnitRange
                Base.IdentityUnitRange
                Base.OneTo
                Base.Slice
                UnitRange
            StepRange
        StepRangeLen
    Base.LogicalIndex
    Base.ReinterpretArray
    Base.ReshapedArray
    Base.SCartesianIndices2
    BitArray
    CartesianIndices
    Core.Compiler.AbstractRange
        Core.Compiler.LinRange
        Core.Compiler.OrdinalRange
            Core.Compiler.AbstractUnitRange
                Core.Compiler.IdentityUnitRange
                Core.Compiler.OneTo
                Core.Compiler.Slice
                Core.Compiler.StmtRange
                Core.Compiler.UnitRange
            Core.Compiler.StepRange
        Core.Compiler.StepRangeLen
    Core.Compiler.BitArray
    Core.Compiler.LinearIndices
    DenseArray
        Array
        Base.CodeUnits
        Base.Experimental.Const
        Random.UnsafeView
        SharedArrays.Sh

In [7]:
# Don't call the function on type: Any

### **Variable Types for Local Variables**

* Global variable doesn't support typing
* Only local varaible can be declared with types. 